# Übung 2 - Polygonzug



In [214]:
# Libraries
import numpy as np
import pandas as pd

In [215]:
#import data
data_Noise = pd.read_csv("./data/data_Noise.txt", delimiter=";")
data_NoNoise = pd.read_csv("./data/data_NoNoise.txt", delimiter=";")
fixpunkte = pd.read_csv("./data/fixpunkte.txt", delimiter=";")
i_meteo = pd.read_csv("./data/i_meteo.txt", delimiter=";")

# const for calcs
T = 12 #°C
P = 1013.25 #hPa
R = 6371000 #m

# "global" funcs
def radian(gons): return (gons * 2 * np.pi) / 400
def gon(rads): return (rads * 400) / 2 * np.pi

data_NoNoise


,Visur_von,Visur_nach,RI,RII,ZI,ZII,D
0,A,P,165.5718,365.5718,103.4797,296.5203,NaN
1,A,PP1,365.7370,165.7370,96.7342,303.2658,27.4199
2,PP1,A,261.8157,61.8157,103.2658,296.7342,27.4199
3,PP1,PP2,60.9770,260.9770,97.2136,302.7864,25.4368
4,PP2,PP1,70.4558,270.4558,102.7864,297.2136,25.4368
5,PP2,PP3,251.6220,51.6220,98.3621,301.6379,33.4681
6,PP3,PP2,371.0396,171.0396,101.6379,298.3621,33.4681
7,PP3,PP4,151.6620,351.6620,99.3118,300.6882,23.3132
8,PP4,PP3,340.9986,140.9986,100.6882,299.3118,23.3132
9,PP4,E,168.9636,368.9636,100.4986,299.5014,44.8211


In [216]:
# 1 Messdatenaufbereitung
def data_preprocessing(df, df_met, df_fix):
    def az_add_or_subtract(az): # adds or subtracts 200 gon to azimut
        if az <= 200:
            return az + 200
        else: return az - 200

    # add columns with info about Standpunkt meteo
    array = pd.DataFrame()
    ## make df for every standpunkt and extract meteo info
    for i in range(len(df["Visur_von"])):
        array = pd.concat([array, df_met[df_met["PNr"] == df["Visur_von"][i]]], axis=0)
    ## concat to main df 
    df = pd.concat([df, array.reset_index()], axis=1).drop(["index", "PNr"], axis=1)
    
    ## Azimuth and zenith angles average
    #data_Noise["R_m_t"] = (data_Noise["RI"]+ data_Noise["RII"] - 200) / 2
    df["R_m"] = (df["RI"] + df["RII"].apply(az_add_or_subtract)) / 2
    df["Z_m"] = (df["ZI"] + 400 - df["ZII"]) / 2

    # init h_a / h_b
    df["h_a"] = 0
    df["h_b"] = 0
    for index, punkt in df_fix.iterrows():
        df["h_a"][df["Visur_von"] == punkt["PNr"]] = float(punkt["H"]) #höhe punkt
    df["h_a"] = df["h_a"] + df["i"] #instrumentenhöhe dazu wo bekannt
    
    # Sanity check (should be small, at max 0.1 otherwise somebody f*cked up)
    #print(df["RI"] - df["RII"].apply(az_add_or_subtract))
    return df


In [217]:
## Distances
def correction_met(d, temp, press): # watchout d in km!!! (pass as m)
    return ((-1 * (temp - T) + 0.3 * (press - P)) * (d / 1000)) / 1000 # genäherte Korrektur

def calculate_h_b(h_a, d_s, zenith, i, t):
    return h_a + d_s * np.cos(radian(zenith)) + i - t

def correction_sehne(h_a, h_b, s):
    return np.sqrt( (s**2 - np.abs(h_a - h_b)) / 
                    ( (1 + (h_a / R)) * (1 + (h_b / R)) ) 
                    )

def correction_factor_distance_flat(X_m):
    return 1 + ((X_m**2) / (2*R**2))

def average(values):
    return sum(values) / len(values)

In [218]:
## Angles
def calc_brechungswinkel(ang_0, ang_1):
    return ang_1 - ang_0

def calculate_richtungswinkel(r_0, beta):
    R = r_0 + 200 + beta

    if R < 0: R = R + 400
    elif R > 400: R = R - 400

    return R

def calculate_richtungsabschluss(coord_0, coord_1):
    return np.arctan2(  (coord_0[1] - coord_1[1]) / 
                        (coord_0[0] - coord_1[0]) 
                        )

## Calculating coordinates (1. GA)
def calculate_next_coordinates(coords_stand, d, az):
    Y = coords_stand[0] + d * np.sin(radian(az))
    X = coords_stand[1] + d * np.cos(radian(az))

In [219]:
def calculate_corrections(df, df_met, df_fix):
    df = data_preprocessing(df, df_met, df_fix)
    temp = df_met['T'].mean()
    press = df_met['p'].mean()

    # calculate meteo correction for every leg with mean meteo meas
    df["Delta_d_met"] = df["D"].apply(correction_met, args=(temp, press))
    ## apply correction for distances
    df["d_m"] = df["D"] - df['Delta_d_met']

    # Lichtweg im Raum
    df["s_m"] = df["d_m"] # genähert

    # Sehne im Raum
    s_0 = list()
    for i, meas in df.iterrows():
        if i == 0 or i == len(data_Noise["D"]) - 1:  # ignore first and last entry
            s_0.append(pd.NA)
            continue
        #print(meas["i"])
        df["h_b"][i] = calculate_h_b(meas["h_a"], meas["s_m"], meas["Z_m"], meas["i"], float(df_met["i"][df_met["PNr"] == meas["Visur_nach"]]))
        #print(h_b)
        s_0.append(correction_sehne(meas["h_a"], df["h_b"][i], meas["s_m"]))
    df["s_0"] = s_0

    # Sehne am Ellipsoid
    df["D_0"] = df["s_0"] # genähert

    #Bogen am Ellipsoid
    X_m = df_fix["X"].mean() - 1200000 # mittlerer Abstand Bern
    correction_factor_flat = correction_factor_distance_flat(X_m)
    df["correction_factor_flat"] = correction_factor_flat
    df["D_flat"] = df["D_0"] * df["correction_factor_flat"]

    # Distanzen Mitteln
    D_m = list()
    D_m.append(pd.NA)
    for i, dist in enumerate(df["D_flat"]):
        if i == 0 or i == len(df["D_flat"]) - 1: continue # ignore first and last entry
        if i % 2 != 0:
            D_m.append((dist + df["D_flat"][i+1]) / 2)
            D_m.append((dist + df["D_flat"][i+1]) / 2)
    D_m.append(pd.NA)
    df["D_m"] = D_m

    return df

data_Noise = calculate_corrections(data_Noise, i_meteo, fixpunkte)
data_Noise

data_NoNoise = calculate_corrections(data_NoNoise, i_meteo, fixpunkte)
data_NoNoise

/var/folders/nb/91ndyn8522572mnr6hfxzk540000gn/T/ipykernel_4693/921075284.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["h_a"][df["Visur_von"] == punkt["PNr"]] = float(punkt["H"]) #höhe punkt
/var/folders/nb/91ndyn8522572mnr6hfxzk540000gn/T/ipykernel_4693/281532403.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["h_b"][i] = calculate_h_b(meas["h_a"], meas["s_m"], meas["Z_m"], meas["i"], float(df_met["i"][df_met["PNr"] == meas["Visur_nach"]]))
/var/folders/nb/91ndyn8522572mnr6hfxzk540000gn/T/ipykernel_4693/921075284.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,Visur_von,Visur_nach,RI,RII,ZI,ZII,D,i,T,p,...,h_a,h_b,Delta_d_met,d_m,s_m,s_0,D_0,correction_factor_flat,D_flat,D_m
0,A,P,165.5718,365.5718,103.4797,296.5203,NaN,1.602,18.0,959.8,...,534.139,0.000000,NaN,NaN,NaN,<NA>,<NA>,1.000033,<NA>,<NA>
1,A,PP1,365.7370,165.7370,96.7342,303.2658,27.4199,1.602,18.0,959.8,...,534.139,535.527029,-0.000596,27.420496,27.420496,27.392875,27.392875,1.000033,27.393766,27.394913
2,PP1,A,261.8157,61.8157,103.2658,296.7342,27.4199,1.620,18.0,959.6,...,1.620,0.231971,-0.000596,27.420496,27.420496,27.39517,27.39517,1.000033,27.396061,27.394913
3,PP1,PP2,60.9770,260.9770,97.2136,302.7864,25.4368,1.620,18.0,959.6,...,1.620,2.816004,-0.000553,25.437353,25.437353,25.413825,25.413825,1.000033,25.414651,25.414653
4,PP2,PP1,70.4558,270.4558,102.7864,297.2136,25.4368,1.537,17.9,959.8,...,1.537,0.340996,-0.000553,25.437353,25.437353,25.41383,25.41383,1.000033,25.414656,25.414653
5,PP2,PP3,251.6220,51.6220,98.3621,301.6379,33.4681,1.537,17.9,959.8,...,1.537,2.475993,-0.000728,33.468828,33.468828,33.454786,33.454786,1.000033,33.455874,33.455877
6,PP3,PP2,371.0396,171.0396,101.6379,298.3621,33.4681,1.459,17.7,959.8,...,1.459,0.520007,-0.000728,33.468828,33.468828,33.454792,33.454792,1.000033,33.45588,33.455877
7,PP3,PP4,151.6620,351.6620,99.3118,300.6882,23.3132,1.459,17.7,959.8,...,1.459,1.487021,-0.000507,23.313707,23.313707,23.313101,23.313101,1.000033,23.313859,23.313858
8,PP4,PP3,340.9986,140.9986,100.6882,299.3118,23.3132,1.683,17.5,959.7,...,1.683,1.654979,-0.000507,23.313707,23.313707,23.3131,23.3131,1.000033,23.313858,23.313858
9,PP4,E,168.9636,368.9636,100.4986,299.5014,44.8211,1.683,17.5,959.7,...,1.683,1.400958,-0.000975,44.822075,44.822075,44.818917,44.818917,1.000033,44.820375,44.818489


In [220]:
# Brechungswinkel
def brechungswinkel(df):
    beta = []
    for i, meas in df.iterrows():
        print(beta)
        if i == len(df) - 1: continue # skip last row (already calculated)
        beta.append(calc_brechungswinkel(df["R_m"][i], df["R_m"][i + 1]))
    df["beta"] = beta

brechungswinkel(data_NoNoise)


[]
[200.16519999999997]
[200.16519999999997, -103.92129999999997]
[200.16519999999997, -103.92129999999997, -200.83870000000002]
[200.16519999999997, -103.92129999999997, -200.83870000000002, 9.478800000000021]
[200.16519999999997, -103.92129999999997, -200.83870000000002, 9.478800000000021, 181.1662]
[200.16519999999997, -103.92129999999997, -200.83870000000002, 9.478800000000021, 181.1662, 119.4176]
[200.16519999999997, -103.92129999999997, -200.83870000000002, 9.478800000000021, 181.1662, 119.4176, -219.37760000000003]
[200.16519999999997, -103.92129999999997, -200.83870000000002, 9.478800000000021, 181.1662, 119.4176, -219.37760000000003, 189.33660000000003]
[200.16519999999997, -103.92129999999997, -200.83870000000002, 9.478800000000021, 181.1662, 119.4176, -219.37760000000003, 189.33660000000003, -172.03500000000003]
[200.16519999999997, -103.92129999999997, -200.83870000000002, 9.478800000000021, 181.1662, 119.4176, -219.37760000000003, 189.33660000000003, -172.03500000000003, 1

ValueError: Length of values (11) does not match length of index (12)